（※このファイルはGoogle Colaboratory用です）

- 途中でURLが表示され、止まることがあります（合計2回）。その場合はURLにアクセスし、キーを入力欄に貼り付けてください。
- アップロードの表示が出たら、音声ファイルを圧縮したzipファイルをアップロードしてください。

In [0]:
# ↓途中で終了した場合にそこから再開するか。新規に作成する場合はFalseにすること
retrain_flag = True
# ---

# リポジトリをクローンし、フォルダ内へ移動
!git clone https://github.com/NON906/NVC_train.git
%cd NVC_train
!git checkout test # DEBUG

# Googleドライブに接続
# 参考：https://qiita.com/tomo_makes/items/b3c60b10f7b25a0a5935
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

# メイン処理開始
import zipfile

from scripts.make_nvm import make_nvm_main
from scripts.gen_targets import gen_targets_main
from scripts.target_train import target_train_main
from scripts.target_pitch_train import target_pitch_train_main
from scripts.target_convert_h5_to_pb import convert_h5_to_pb_main
from scripts.make_nvz import make_nvz_main

targets_dir = 'targets'
outputs_dir = 'drive/nvc'
nvz_file = 'drive/nvc/target.nvz'

retrain_step = 0
if retrain_flag and os.path.isfile(outputs_dir + '/retrain.dat'):
    retrain_step = int(open(outputs_dir + '/retrain.dat', 'r').read())
else:
    if os.path.isdir(outputs_dir):
        shutil.rmtree(outputs_dir)
    if os.path.isdir('gen_targets'):
        shutil.rmtree('gen_targets')

if retrain_step < 2:
    # soxのインストール
    !apt-get -y install sox

    # SPTKのインストール
    !apt-get -y install csh
    !wget http://downloads.sourceforge.net/sp-tk/SPTK-3.11.tar.gz
    !tar xfvz SPTK-3.11.tar.gz
    %cd SPTK-3.11
    !./configure
    !make
    !make install
    %cd ..
  
    # 音声の入ったzipファイルをアップロード
    from google.colab import files
    import os

    uploaded = files.upload()

    for fn in uploaded.keys():
        os.rename(fn, 'targets.zip')

    with zipfile.ZipFile('targets.zip') as existing_zip:
        existing_zip.extractall(targets_dir)

if retrain_step < 1:
    make_nvm_main(targets_dir, outputs_dir + '/target.nvm')
    open(outputs_dir + '/retrain.dat', 'w').write('1')

if retrain_step < 2:
    gen_targets_main(targets_dir, 'gen_targets', outputs_dir + '/gen_targets.zip')
    open(outputs_dir + '/retrain.dat', 'w').write('2')
else:
    with zipfile.ZipFile(outputs_dir + '/gen_targets.zip') as existing_zip:
        existing_zip.extractall('gen_targets')

if retrain_step < 3:
    if not os.path.isfile(outputs_dir + '/target.h5'):
        target_train_main('gen_targets', outputs_dir + '/target.h5', 20, -1, 32)
    else:
        target_train_main('gen_targets', outputs_dir + '/target.h5', 20, -1, 32, outputs_dir + '/target.h5')
    open(outputs_dir + '/retrain.dat', 'w').write('3')

if retrain_step < 4:
    if not os.path.isfile(outputs_dir + '/pitch.h5'):
        target_pitch_train_main('gen_targets', outputs_dir + '/pitch.h5', 20, -1, 32)
    else:
        target_pitch_train_main('gen_targets', outputs_dir + '/pitch.h5', 20, -1, 32, outputs_dir + '/pitch.h5')
    open(outputs_dir + '/retrain.dat', 'w').write('4')

# 新しいバージョンだと、TensorFlowSharpでエラーが出るので、古いバージョンを用意する
!pip install tensorflow-gpu==1.5.0
!apt-get install cuda-9.0
    
if retrain_step < 5:
    convert_h5_to_pb_main(outputs_dir + '/target.h5', outputs_dir + '/target.pb', 'target_dense_f/BiasAdd')
    open(outputs_dir + '/retrain.dat', 'w').write('5')

if retrain_step < 6:
    convert_h5_to_pb_main(outputs_dir + '/pitch.h5', outputs_dir + '/pitch.pb', 'pitch_relu_f/Relu')
    open(outputs_dir + '/retrain.dat', 'w').write('6')

if retrain_step < 7:
    make_nvz_main(nvz_file, outputs_dir + '/target.nvm', outputs_dir + '/target.pb', outputs_dir + '/pitch.pb')
    open(outputs_dir + '/retrain.dat', 'w').write('7')

# nvzファイルのダウンロード
from google.colab import files
files.download(nvz_file)